### Run tensoflow lite model through frames from panorama video

In [ ]:
# see https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb#scrollTo=6cPY9Ou4sWs_

In [2]:
import tensorflow as tf

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import time

import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 500)

### Helper functions for downloading images and for visualization.

Visualization code adapted from [TF object detection API](https://github.com/tensorflow/models/blob/master/research/object_detection/utils/visualization_utils.py) for the simplest required functionality.

In [3]:
def display_image(image):
    fig = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)


def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
    _, filename = tempfile.mkstemp(suffix=".jpg")
    response = urlopen(url)
    image_data = response.read()
    image_data = BytesIO(image_data)
    pil_image = Image.open(image_data)
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert("RGB")
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    print("Image downloaded to %s." % filename)
    if display:
        display_image(pil_image)
    return filename


def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = bottom + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str,
                  fill="black",
                  font=font)
    text_bottom -= text_height - 2 * margin

def is_big_box(ymin, xmin, ymax, xmax):
    if (xmax - xmin >= 0.25) and (ymax - ymin >= 0.25):
        return True
    else:
        return False
    
font_path = '/usr/share/fonts/truetype/dejavu/DejaVuSansMono.ttf'
# orig_font_path = '"/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf"'
def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.1, filter_big_boxes = False):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())
    try:
        font = ImageFont.truetype(font_path,
                              35)
    except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()
    num_iter = min(boxes.shape[0], max_boxes)
    for i in range(num_iter):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = tuple(boxes[i])
            if filter_big_boxes and not is_big_box(ymin, xmin, ymax, xmax):
                continue
            
            class_id = int(class_names[i])
            class_name = labels[class_id+1]
            class_str = f'{class_name}({class_id})'

            display_str = "{}: {}%".format(str(class_str),
                                         int(100 * scores[i]))

            # color = colors[hash(class_names[i]) % len(colors)]
            # bad for trucks, fix
            color = colors[5]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
            draw_bounding_box_on_image(
              image_pil,
              ymin,
              xmin,
              ymax,
              xmax,
              color,
              font,
              display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
    return image

### Apply tflite module

In [4]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img

In [5]:
# model_path = 'ourmodels/coco_ssd_mobilenet_v1_1.0_quant_2018_06_29'
model_path = 'ourmodels/coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/detect.tflite'
label_path = 'ourmodels/coco_ssd_mobilenet_v1_1.0_quant_2018_06_29/labelmap.txt'

In [6]:
def load_labels(filename):
    with open(filename, 'r') as f:
        return [line.strip() for line in f.readlines()]
labels = load_labels(label_path)

In [7]:
# The TFLite_Detection_PostProcess custom op node has four outputs
# detection_boxes: a tensor of shape [1, num_boxes, 4] with normalized coordinates
# detection_classes: a tensor of shape [1, num_boxes] containing class prediction for each box
# detection_scores: a tensor of shape [1, num_boxes]
# num_boxes: a tensor of size 1 containing the number of detected boxes
# see https://stackoverflow.com/questions/59143641/how-to-get-useful-data-from-tflite-object-detection-python
# and https://github.com/tensorflow/tensorflow/issues/34761
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
def get_tflite_results(img_path):
    print(img_path)
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]
    # img_path = '/var/tmp/tmpjcmgv9u0.jpg'
    img = Image.open(img_path).resize((width, height))
    input_data = np.expand_dims(img, axis = 0)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    detection_boxes = interpreter.get_tensor(output_details[0]['index'])
    detection_classes = interpreter.get_tensor(output_details[1]['index'])
    detection_scores = interpreter.get_tensor(output_details[2]['index'])
    num_boxes = interpreter.get_tensor(output_details[3]['index'])
    print(num_boxes)
    for i in range(int(num_boxes[0])):
        if detection_scores[0, i] > .5:
            x = detection_boxes[0, i, [1, 3]] * width
            y = detection_boxes[0, i, [0, 2]] * height
            rectangle = [x[0], y[0], x[1], y[1]]
            class_id = detection_classes[0, i]
            print(class_id)
            print(labels[int(class_id+1)])
#             if class_id == 66.0:
#                 print(rectangle)
    return detection_boxes, detection_classes, detection_scores            

In [8]:
def debug_bad_photo(img_path, detection_boxes, detection_classes, detection_scores, filter_big_boxes = True):
    img = load_img(img_path)

    image_with_boxes = draw_boxes(
        img.numpy(),
        detection_boxes[0],
        detection_classes[0], detection_scores[0],
        filter_big_boxes = filter_big_boxes
    )

    display_image(image_with_boxes)

In [68]:
CAR_CLASS = 2
TRUCK_CLASS = 7
BUS_CLASS = 5 # sometimes returns for trucks
ALLOWED_CLASSES = set([CAR_CLASS, TRUCK_CLASS, BUS_CLASS])
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def check_box_big_enough(class_id, share_of_width, share_of_height):
#     min_share_of_screen_cars = 0.22 # for cars
    # 'images_from_video/videos_ok_618_0ac_6180ac75-f84e-4f97-a93a-cd5ab073ffc1' - cars are lower
    min_share_of_width_cars = 0.22 # for cars
    min_share_of_height_cars = 0.18
    
    min_share_of_width_trucks = 0.15
    min_share_of_height_trucks = 0.25
    if class_id == CAR_CLASS:
        return share_of_width >= min_share_of_width_cars and share_of_height >= min_share_of_height_cars
    else:
        return share_of_width >= min_share_of_width_trucks and share_of_height >= min_share_of_height_trucks
    
def check_big_car_tflite_results(img_path, debug_all = False, debug_filter_big_boxes = True,
                                debug_bounding_box_margin = False,
                                debug_share_of_screen = False,
                                print_bad_photo = True,
                                no_debug_message = False,
                                images_debug = None):
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]
    safe_margin = width/60
    img = Image.open(img_path).resize((width, height))
    input_data = np.expand_dims(img, axis = 0)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    detection_boxes = interpreter.get_tensor(output_details[0]['index'])
    detection_classes = interpreter.get_tensor(output_details[1]['index'])
    detection_scores = interpreter.get_tensor(output_details[2]['index'])
    num_boxes = interpreter.get_tensor(output_details[3]['index'])
    big_car_found = False
    boxes_ok = False
    big_box_not_in_the_center = False
    box_outside_of_image = False
    for i in range(int(num_boxes[0])):
        class_id = detection_classes[0, i]

        # for low probabilites it might me part of the car/truck in most cases
        if detection_scores[0, i] >= .5 and class_id in ALLOWED_CLASSES:
            x = detection_boxes[0, i, [1, 3]] * width
            y = detection_boxes[0, i, [0, 2]] * height
            rectangle = [x[0], y[0], x[1], y[1]]
            
            # check box has big size
            # 4.22 for trucks: 'images_from_video/videos_failed_436_d36_436d36aa-393d-4881-9fe9-e80016d38232/145.jpg'
            # see car with such share images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1273.jpg
            min_share_of_screen = 0.22
            
            share_of_width = (x[1] - x[0])/width
            share_of_height = (y[1] - y[0])/height
            if check_box_big_enough(class_id, share_of_width, share_of_height):

                # check box is central (1.mp4 has big bounding box not in the center)
                if (x[0] < width/2) and (x[1] > width/2) and (y[0] < height/2) and (y[1] > height/2):
                    big_car_found = True
                    if (x[0] > safe_margin) and (y[0] > safe_margin) and (width - x[1] > safe_margin) and (height - y[1] > safe_margin):
                        boxes_ok = True
                    else:
                        if (x[0] <= 0) or (y[0] <= 0) or (width - x[1] <= 0) or (height - y[1] <= 0):
                            box_outside_of_image = True
                            if debug_bounding_box_margin:
                                if not no_debug_message:
                                    print(f'bounding box outside of image: {rectangle}')
                        else:
                            if debug_bounding_box_margin:
                                if not no_debug_message:
                                    print(f'bounding box without margin: {rectangle}')
                else:
                    big_box_not_in_the_center = True
            else:
                # check box is central (1.mp4 has big bounding box not in the center)
                if (x[0] < width/2) and (x[1] > width/2) and (y[0] < height/2) and (y[1] > height/2):
                    if debug_share_of_screen:
                        if not no_debug_message:
                            print(f"Share of screen to small for central box, share_of_width: {share_of_width}, share of height: {share_of_height}")
                
            
    if big_car_found and boxes_ok:
        if debug_all:
            print(f"ok frame: printing {img_path}")
            debug_bad_photo(img_path, detection_boxes, detection_classes, detection_scores, debug_filter_big_boxes)

    else: 
        if big_box_not_in_the_center:
            if not no_debug_message:
                print(f"big box not in the center found: {img_path}")
        if not big_car_found:
            if not no_debug_message:
                print(f"no car/truck/bus big enough found: {img_path}")
        if not boxes_ok:
            if not no_debug_message:
                print(f"bounding box not ok: {img_path}")
        if print_bad_photo:
            debug_bad_photo(img_path, detection_boxes, detection_classes, detection_scores, debug_filter_big_boxes)
        if images_debug != None:
            images_debug.append(img_path)
    return big_car_found, boxes_ok, box_outside_of_image

In [10]:
# path = 'images_from_video/1.mp4/121.jpg'
# big_car_found, boxes_ok, box_outside_of_image = check_big_car_tflite_results(path, debug_all=True, debug_filter_big_boxes = False)
# big_car_found, boxes_ok, box_outside_of_image

### analyze real streams from auto.ru data

In [11]:
# !mkdir images_from_video

In [12]:
# !mkdir images_from_video/1.mp4
# !ffmpeg -i 1.mp4 images_from_video/1.mp4/%d.jpg

In [13]:
import glob

In [14]:
# images_from_video/videos_ok_211_0bb_2110bb80-da68-46f3-b77a-a4eeb4ac8724
COMPLETED = 'completed'
FAILED = 'failed'
S3_PREFIX = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/'

# frames were generated in http://vgorovoy-01-dev.sas.yp-c.yandex.net:8889/notebooks/panorama/analyze_panoramas_rate.ipynb
def get_url_status_from_path(path):
    if 'videos_ok' in path:
        path = path.replace('images_from_video/videos_ok_', '')
        status = COMPLETED
    elif 'videos_failed' in path:
        path = path.replace('images_from_video/videos_failed_', '')
        status = FAILED
    else:
        raise Exception(f'Strange folder path passed: {path}')
        
    url = S3_PREFIX + path.replace('_', '/')
    return url, status

In [59]:
BAD = 'BAD'
GOOD = 'GOOD'
UNKNOWN = 'UNKNOWN'
def check_video(file_path, make_frames = True, debug_all = True, 
                debug_filter_big_boxes = True, folder_path = '', 
                debug_margins = False,
                print_bad_photo = True,
                debug_share_of_screen = False,
                no_debug_message = False,
                images_debug = None
               ):
    if folder_path == '':
        file_path_upload = file_path.replace('/', '_')
        if make_frames:
            !mkdir images_from_video/$file_path_upload
            !ffmpeg -i $file_path images_from_video/$file_path_upload/%d.jpg
        # fails for https://autoru-panorama-internal.s3.mds.yandex.net/upload/612/b58/612b589e-f752-441e-b8ef-b3c628f734cd
        # but only on ubuntu - on mac everything works
        # advice on stackoverflow didn't help https://stackoverflow.com/questions/30782771/what-does-past-duration-x-xxx-too-large-mean/50432919#50432919
        folder_path = 'images_from_video/' + file_path_upload
    all_files = glob.glob(folder_path + '/*.jpg')
    ok_count = 0
    bad_count = 0
    ok_box_count = 0
    no_big_vehicle_middle_screen_found = 0
    bounding_boxes_outside_image = 0
    total_analyzed_frames = 0
    bad_box_count = 0
    for f in sorted(all_files):
        c_frame = int(f.split('/')[-1].split('.')[0])
        if c_frame%12 == 1: # analyze one frame in 0.5 seconds
            is_ok_frame, is_ok_box, box_outside_of_image = check_big_car_tflite_results(f, 
                                                                  debug_all, 
                                                                  debug_filter_big_boxes=debug_filter_big_boxes,
                                                                  debug_bounding_box_margin = debug_margins,
                                                                  print_bad_photo = print_bad_photo,
                                                                  debug_share_of_screen = debug_share_of_screen,
                                                                  no_debug_message = no_debug_message,
                                                                  images_debug = images_debug
                                                                 )
            total_analyzed_frames += 1
            if is_ok_frame:
                ok_count += 1
            else:
                no_big_vehicle_middle_screen_found += 1
            if not is_ok_box:
                bad_box_count += 1
            if box_outside_of_image:
                bounding_boxes_outside_image += 1
    if not no_debug_message:
        print(f'ok frames: {ok_count}')
        print(f'bad frames without class: {no_big_vehicle_middle_screen_found}')
        print(f'bad boxes: {bad_box_count}')
        print(f'bad boxes outside of image: {bounding_boxes_outside_image}')

    car_found_but_bounding_box_unsafe_margin = bad_box_count - no_big_vehicle_middle_screen_found
    bad_frames = (no_big_vehicle_middle_screen_found + car_found_but_bounding_box_unsafe_margin)
    really_bad_frames = (no_big_vehicle_middle_screen_found + bounding_boxes_outside_image)
    if total_analyzed_frames == 0:
        if not no_debug_message:
            print(f'no frames found: {folder_path}')
        share_no_big_vehicle_middle_screen_found = 2
        share_car_found_but_bounding_box_unsafe_margin = 2
        share_bad_frames = 2
        share_really_bad_frames = 2
    else:
        share_no_big_vehicle_middle_screen_found = no_big_vehicle_middle_screen_found/total_analyzed_frames
        share_bad_frames = bad_frames/total_analyzed_frames
        share_really_bad_frames = really_bad_frames/total_analyzed_frames
        if total_analyzed_frames - no_big_vehicle_middle_screen_found != 0:
            share_car_found_but_bounding_box_unsafe_margin = car_found_but_bounding_box_unsafe_margin/(total_analyzed_frames - no_big_vehicle_middle_screen_found)
        else:
            share_car_found_but_bounding_box_unsafe_margin = 2

    
    if ok_count > 30 and share_really_bad_frames <= 0.05:
        result = GOOD
    elif share_really_bad_frames >= 0.16:
        result = BAD
    else:
        result = UNKNOWN
    url, status = get_url_status_from_path(folder_path)
    res_dict = {
        'url': url,
        'panorama_status': status,
        'image_path': folder_path,
        'classifier_result': result,
        'total_analyzed_frames': total_analyzed_frames,
        'big_vehicle_middle_screen_found': ok_count,
        'no_big_vehicle_middle_screen_found': bad_count,
        'car_found_but_bounding_box_unsafe_margin': car_found_but_bounding_box_unsafe_margin, 
        'bounding_boxes_outside_image': bounding_boxes_outside_image,
        'bad_frames': bad_frames,
        'really_bad_frames': really_bad_frames,
        'share_no_big_vehicle_middle_screen_found': share_no_big_vehicle_middle_screen_found,
        'share_car_found_but_bounding_box_unsafe_margin': share_car_found_but_bounding_box_unsafe_margin,
        'share_bad_frames': share_bad_frames,
        'share_really_bad_frames': share_really_bad_frames
    }
    return res_dict
    

In [16]:
def get_filepath_from_url(url):
    name = url.replace('https://autoru-panorama-internal.s3.mds.yandex.net/upload/', '').replace('/', '_')
    filepath = 'videos_failed/' + name
    return filepath

In [17]:
# check_big_car_tflite_results('images_from_video/videos_failed_436_d36_436d36aa-393d-4881-9fe9-e80016d38232/145.jpg')

In [21]:
path = 'images_from_video/videos_ok_71e_024_71e024f6-5b4d-48b6-9755-ba9c89aaf218'
path = 'images_from_video/videos_failed_d09_b98_d09b9863-4a2d-498e-a999-a74aec917a8c'
check_video(file_path = '', make_frames = False, 
                            debug_all = False, 
                            debug_filter_big_boxes = False, folder_path = path,
                            debug_margins = False,
                            print_bad_photo=False,
                            debug_share_of_screen=False,
                            no_debug_message=True
                           )

{'url': 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/d09/b98/d09b9863-4a2d-498e-a999-a74aec917a8c',
 'panorama_status': 'failed',
 'image_path': 'images_from_video/videos_failed_d09_b98_d09b9863-4a2d-498e-a999-a74aec917a8c',
 'classifier_result': 'BAD',
 'total_analyzed_frames': 94,
 'big_vehicle_middle_screen_found': 0,
 'no_big_vehicle_middle_screen_found': 0,
 'car_found_but_bounding_box_unsafe_margin': 0,
 'bounding_boxes_outside_image': 0,
 'bad_frames': 94,
 'really_bad_frames': 94,
 'share_no_big_vehicle_middle_screen_found': 1.0,
 'share_car_found_but_bounding_box_unsafe_margin': 2,
 'share_bad_frames': 1.0,
 'share_really_bad_frames': 1.0}

### Analysis of sample failed videos

In [ ]:
# analyze failed images
# check_video(file_path='2.mp4')# ok


# 1 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/612/b58/612b589e-f752-441e-b8ef-b3c628f734cd
# снимал близко слишком, часто машина обрезана (UNK_ERROR)
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/612/b58/612b589e-f752-441e-b8ef-b3c628f734cd' # failed ffmpeg
# ok result
# ok frames: 80
# bad frames: 13
# ok boxes: 23
# bad boxes: 70

# 2 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/436/d36/436d36aa-393d-4881-9fe9-e80016d38232
# в ноутбуке не работает 
# грузовик без кузова (может не дошел пару метров)
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/436/d36/436d36aa-393d-4881-9fe9-e80016d38232'
# small boxes should be used for trucks
# ok frames: 112
# bad frames: 12
# ok boxes: 112
# bad boxes: 12
# todo make smaller frames

# 3 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/986/879/986879c4-584d-44d6-89e5-1aa0f37a5983
# грузовик без кузова - нормальное видео, не понимаю почему не собралось (BIG_STEP_IN_RECONSTRUCTION)
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/986/879/986879c4-584d-44d6-89e5-1aa0f37a5983'
# small boxes should be used for trucks
# ok frames: 160
# bad frames: 19
# ok boxes: 160
# bad boxes: 19


# 4 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/6e9/354/6e935483-52d0-4c1f-9079-8876fe77b503
# в ноутбуке не работает
# легковушка нормально снятая
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/6e9/354/6e935483-52d0-4c1f-9079-8876fe77b503'
# too small box for car
# ok frames: 96
# bad frames: 2
# ok boxes: 96
# bad boxes: 2
# check_video(filepath, make_frames=False, debug_all = False)
# check_video(filepath, make_frames=True, debug_all = False)

# 5 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/be6/cce/be6ccecb-b0c8-4e63-b02b-43e5317e4fc9
# в ноутбуке работает, но тяжелое видео
# нормально снято (из проблем только тень) (UNK_ERROR)
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/be6/cce/be6ccecb-b0c8-4e63-b02b-43e5317e4fc9'
# error in box detection
# ok frames: 303
# bad frames: 0
# ok boxes: 299
# bad boxes: 4

# 6 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/ef4/c47/ef4c4767-a1ce-4b3d-9c0e-30ab4f88d61e
# обрезанная машина, снимает прямо у забора
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/ef4/c47/ef4c4767-a1ce-4b3d-9c0e-30ab4f88d61e'
# some erros while uploading
# ok frames: 100
# bad frames: 59
# ok boxes: 17
# bad boxes: 142

# 7 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/d98/a9f/d98a9f97-7491-4bed-8b32-acb49de28be8
# нормально снята лекговушка (UNK_ERROR)
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/d98/a9f/d98a9f97-7491-4bed-8b32-acb49de28be8'
# train for Niva!
# ok frames: 144
# bad frames: 3
# ok boxes: 144
# bad boxes: 3

# 8 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/831/493/83149371-205e-4b53-aadf-debf44d16518
# легковушка, нормальна снята
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/831/493/83149371-205e-4b53-aadf-debf44d16518'
# ok, just close to the border
# ok frames: 166
# bad frames: 0
# ok boxes: 162
# bad boxes: 4

# 9 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/55d/8ba/55d8bae0-5fd8-4c32-84a8-4c7973021892
# обрезана в кадре (UNK_ERROR)
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/55d/8ba/55d8bae0-5fd8-4c32-84a8-4c7973021892'
# ok frames: 76
# bad frames: 6
# ok boxes: 33
# bad boxes: 49

# 0 - https://autoru-panorama-internal.s3.mds.yandex.net/upload/3d5/714/3d5714fc-6ef6-4479-8d83-4140312eeaf2
# posting from screen (CHECK_NORMALIZATION)
url = 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/3d5/714/3d5714fc-6ef6-4479-8d83-4140312eeaf2'
# столб
# ok frames: 133
# bad frames: 20
# ok boxes: 133
# bad boxes: 20

filepath = get_filepath_from_url(url)
check_video(filepath, make_frames=True, debug_all = False, debug_filter_big_boxes = False)
get_filepath_from_url(url)

### Analysis of sample completed videos

In [ ]:
all_folders = glob.glob('images_from_video/*')
i = 0
for folder in all_folders:
    
    if 'videos_ok' in folder:
        i += 1
#         print(folder)
        if i == 1:
            print(folder)
            res = check_video(file_path = '', make_frames = False, 
                        debug_all = False, 
                        debug_filter_big_boxes = False, folder_path = folder,
                        debug_margins = True,
                        print_bad_photo=False
                       )
            print(res)
#         break

# images_from_video/videos_ok_e86_89d_e8689d6f-1bf9-4b83-a592-82ecf4956e83
# ok frames: 80
# bad frames: 0
# ok boxes: 72
# bad boxes: 8
# after fix;
# ok frames: 80
# bad frames without class: 0
# bad boxes: 7
# bad boxes outside of image: 2

# images_from_video/videos_ok_287_b38_287b380e-e940-44d1-bf0d-e4e1401dd015
# suitcase instead of car
# ok frames: 154
# bad frames: 3
# ok boxes: 154
# bad boxes: 3

# сильно обрезанные тоже склеивают
# images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7
# ok frames: 82
# bad frames: 7
# ok boxes: 39
# bad boxes: 50
# after fix:
# ok frames: 82
# bad frames without class: 7
# bad boxes: 44
# bad boxes outside of image: 12

# images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df i = 4
# один раз не нашли машину, остальное слишком маленький фрагмент экрана
# ok frames: 136
# bad frames: 8
# ok boxes: 136
# bad boxes: 8
# after fixing:
# ok frames: 143
# bad frames without class: 1
# bad boxes: 1
# bad boxes outside of image: 0

# debugging photo: images_from_video/videos_ok_da4_dcd_da4dcd26-08af-4ac9-b233-775f34c89bce/1381.jpg
# truck 48% 1 image
# ok frames: 126
# bad frames: 1
# ok boxes: 126
# bad boxes: 1

# images_from_video/videos_ok_e13_766_e1376683-cec9-440b-b5ef-ce831686f60d
# ok frames: 190
# bad frames: 0
# ok boxes: 190
# bad boxes: 0

# images_from_video/videos_ok_03d_04a_03d04ae5-1954-426c-b6ed-2c997f3ddcca -7
# train on 2 images, car 47% on one, bad boxes sometimes really true
# ok frames: 75
# bad frames: 3
# ok boxes: 49
# bad boxes: 29
# after fixes
# ok frames: 75
# bad frames without class: 3
# bad boxes: 24
# bad boxes outside of image: 7


# images_from_video/videos_ok_89b_d79_89bd793d-2069-421d-beff-7a93c8874a95
# bounding box not ok: images_from_video/videos_ok_89b_d79_89bd793d-2069-421d-beff-7a93c8874a95/1153.jpg
# too big bounding box
# ok frames: 179
# bad frames: 0
# ok boxes: 178
# bad boxes: 1
# after fixes
# ok frames: 179
# bad frames without class: 0
# bad boxes: 1
# bad boxes outside of image: 1

# images_from_video/videos_ok_31e_57f_31e57f3e-a6fb-475f-a204-0fd9b8250be4
# was 1 bad box - but fixed by fixing margin
# ok frames: 164
# bad frames: 0
# ok boxes: 163
# bad boxes: 1 (fixed - 0)

# images_from_video/videos_ok_46a_061_46a061a5-b242-49b1-a41e-65f13b003784 - 10
# idea # strictly out of picture boxes
# ok frames: 106
# bad frames: 0
# ok boxes: 66
# bad boxes: 40
# after fixes
# ok frames: 106
# bad frames without class: 0
# bad boxes: 40
# bad boxes outside of image: 12

# images_from_video/videos_ok_a4f_99d_a4f99db1-7605-449f-9b9f-c1ef9d0f2f7f - 11
# ok frames: 223
# bad frames without class: 2
# bad boxes: 46
# bad boxes outside of image: 22


# images_from_video/videos_ok_4c7_4ef_4c74ef4b-6385-4fe1-b8e5-3fa6b94e5039 - 12
# ok frames: 174
# bad frames without class: 0
# bad boxes: 0
# bad boxes outside of image: 0

# images_from_video/videos_ok_62d_768_62d768ce-077c-471d-bf25-22cb5cf6767d - 13
# cell_phone bug with car, a lot of times outside the boxes
# 
# ok frames: 55
# bad frames without class: 17
# bad boxes: 40
# bad boxes outside of image: 6

# images_from_video/videos_ok_03c_73d_03c73d60-43c5-4142-b4f4-c040eeeb862f
# ok frames: 175
# bad frames without class: 2
# bad boxes: 11
# bad boxes outside of image: 2


# images_from_video/videos_ok_211_0bb_2110bb80-da68-46f3-b77a-a4eeb4ac8724
# ok frames: 181
# bad frames without class: 0
# bad boxes: 0
# bad boxes outside of image: 0

### Process all videos

In [33]:
def process_all_videos():
    all_folders = glob.glob('images_from_video/*')
    all_results = []
    i = 0
    for folder in all_folders:
    
        if 'videos_ok' in folder or 'videos_failed' in folder:
#             print(f"processing {folder}")
            res = check_video(file_path = '', make_frames = False, 
                        debug_all = False, 
                        debug_filter_big_boxes = False, folder_path = folder,
                        debug_margins = False,
                        print_bad_photo=False,
                        no_debug_message=True
                       )
#             print(res)
            all_results.append(res)
    return all_results

In [34]:
final_results = process_all_videos()

In [35]:
!curl --data "text=Finished classifying panorama videos with tflite @VladimirGorovoy&parse_mode=Markdown" https://api.telegram.org/bot345259261:AAGBS5bxlxTe8I9cp_3mCpP-tghLHAuAOkw/sendMessage?chat_id=-233546442

{"ok":true,"result":{"message_id":12988,"from":{"id":345259261,"is_bot":true,"first_name":"RealtyCallCenterBot","username":"RealtyCallCenterBot"},"chat":{"id":-233546442,"title":"BotTests","type":"group","all_members_are_administrators":true},"date":1585908743,"text":"Finished classifying panorama videos with tflite @VladimirGorovoy","entities":[{"offset":49,"length":16,"type":"mention"}]}}

### Analyse final results of classifier

In [36]:
df_results = pd.DataFrame(final_results)

In [50]:
df_results.to_csv('classification_results_v2.tsv', sep = '\t', index = False)

In [37]:
pd.crosstab(df_results.classifier_result, [df_results.panorama_status])

panorama_status,completed,failed
classifier_result,,
BAD,27,31
GOOD,184,55
UNKNOWN,34,19


### 2nd version anlaysis
share of reall bad photos > 15%

In [72]:
import IPython.display

In [67]:
def check_videos(paths):
    all_images_debug = []
    for folder in paths:
        images_debug = []
        print(f"processing video: {folder}")
        res = check_video(file_path = '', make_frames = False, 
                        debug_all = False, 
                        debug_filter_big_boxes = False, folder_path = folder,
                        debug_margins = False,
                        print_bad_photo=False,
                        no_debug_message=True,
                        images_debug = images_debug
                       )
        all_images_debug.append(images_debug)
        print(res)
    return all_images_debug

In [44]:
s_bad_completed = df_results[(df_results.classifier_result == BAD) & (df_results.panorama_status == COMPLETED)].sample(5)

In [60]:
s_bad_failed = df_results[(df_results.classifier_result == BAD) & (df_results.panorama_status == FAILED)].sample(5)

In [92]:
s_unknown_completed  = df_results[(df_results.classifier_result == UNKNOWN) & (df_results.panorama_status == FAILED)].sample(5)

In [96]:
s_unknown_failed = s_unknown_completed

In [97]:
unknown_failed_images = unknown_completed_images

In [99]:
s_unknown_completed  = df_results[(df_results.classifier_result == UNKNOWN) & (df_results.panorama_status == COMPLETED)].sample(5)

In [102]:
len(df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == COMPLETED)])

184

In [103]:
len(df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == COMPLETED) & (df_results.bad_frames > 0)])

93

In [104]:
s_ok_completed = df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == COMPLETED) & (df_results.bad_frames > 0)].sample(10)

In [109]:
len(df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == FAILED)])

55

In [110]:
len(df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == FAILED) & (df_results.bad_frames > 0)])

32

In [111]:
s_ok_failed = df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == FAILED) & (df_results.bad_frames > 0)].sample(10)

In [91]:
bad_failed_images = all_images_debug

In [112]:
# check_videos(list(s_bad_completed.image_path))
# all videos trash
# all_images_debug = check_videos(list(s_bad_failed.image_path))
# all videos trash

# unknown_completed_images = check_videos(list(s_unknown_completed.image_path))
# was failed, fixed
# good trucks, some cars with low border

# unknown_completed_images = check_videos(list(s_unknown_completed.image_path))
# 2 ok, 3 not very good

# ok_completed_images = check_videos(list(s_ok_completed.image_path))
# 4 of 10 not very good


# ok_failed_images = check_videos(list(s_ok_failed.image_path))
# 5 of 5 ok






processing video: images_from_video/videos_failed_76e_dc9_76edc944-4c2b-4b0b-8847-63d72cfee674
{'url': 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/76e/dc9/76edc944-4c2b-4b0b-8847-63d72cfee674', 'panorama_status': 'failed', 'image_path': 'images_from_video/videos_failed_76e_dc9_76edc944-4c2b-4b0b-8847-63d72cfee674', 'classifier_result': 'GOOD', 'total_analyzed_frames': 176, 'big_vehicle_middle_screen_found': 172, 'no_big_vehicle_middle_screen_found': 0, 'car_found_but_bounding_box_unsafe_margin': 0, 'bounding_boxes_outside_image': 0, 'bad_frames': 4, 'really_bad_frames': 4, 'share_no_big_vehicle_middle_screen_found': 0.022727272727272728, 'share_car_found_but_bounding_box_unsafe_margin': 0.0, 'share_bad_frames': 0.022727272727272728, 'share_really_bad_frames': 0.022727272727272728}
processing video: images_from_video/videos_failed_f7f_3cc_f7f3cc8a-01bd-4902-aaac-0d949a0718b9
{'url': 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/f7f/3cc/f7f3cc8a-01bd-4902-aaac

In [114]:
# IPython.display.Image(all_images_debug[0][0])
# IPython.display.Image(all_images_debug[0][10])
# IPython.display.Image(all_images_debug[0][20])
# IPython.display.Image(all_images_debug[0][30])


# all_images_debug = unknown_failed_images

#  3 кривые, 2 норм
# all_images_debug = unknown_completed_images

# all_images_debug = ok_completed_images
# 4 of 10 подрезаны или столб или мутные

all_images_debug = ok_failed_images
# 5 of 5 good

# almost all good
# commented to keep notebook size small
# for i in range(len(all_images_debug)):
#     print(i)
#     images = np.random.choice(all_images_debug[i], min(10, len(all_images_debug[i])))
#     print(images)
#     display(*map(IPython.display.Image, images))

# all_images_debug[0].sample(10)

In [88]:
i = 0
np.random.choice(all_images_debug[i], min(10, len(all_images_debug[i])))

array(['images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/133.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/721.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/709.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/709.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/49.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/145.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/553.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/313.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/145.jpg',
       'images_from_video/videos_failed_d86_97c_d8697cde-479a-4984-91f5-ff022f773fdc/73.jpg'],
      dtype='<U85')

In [82]:
map(IPython.display.Image, all_images_debug[0])

### 1st version analysis
just > 20 bad images is bad

In [ ]:
df_results[(df_results.classifier_result == BAD) & (df_results.panorama_status == COMPLETED)]

In [ ]:
# len(df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == COMPLETED) 
#            & (df_results.share_bad_frames > 0.1)])
# 29
df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == COMPLETED) 
           & (df_results.share_bad_frames > 0.3)]

In [ ]:
df_results[(df_results.classifier_result == GOOD) & (df_results.panorama_status == COMPLETED) 
           & (df_results.share_really_bad_frames > 0.15)]

In [ ]:
print(len(df_results[(df_results.panorama_status == FAILED) 
           & (df_results.share_really_bad_frames > 0.15)]))

In [ ]:
print(len(df_results[(df_results.panorama_status == FAILED) 
           & (df_results.share_really_bad_frames < 0.05)]))

In [ ]:
list(df_results[(df_results.panorama_status == FAILED) 
           & (df_results.share_really_bad_frames < 0.05)].sample(5).image_path)

In [ ]:
list(df_results[(df_results.panorama_status == FAILED) 
           & (df_results.share_really_bad_frames > 0.15)].sample(5).image_path)

In [ ]:
list(df_results[(df_results.panorama_status == COMPLETED) 
           & (df_results.share_really_bad_frames > 0.15)].image_path)

In [ ]:
list(df_results[(df_results.panorama_status == FAILED) 
           & (df_results.share_really_bad_frames < 0.05)].sample(5).image_path)

In [ ]:
print(len(df_results[(df_results.panorama_status == FAILED) 
           & (df_results.share_really_bad_frames > 0.05) & (df_results.share_really_bad_frames <= 0.15)]))

In [ ]:
path = 'images_from_video/videos_ok_71e_024_71e024f6-5b4d-48b6-9755-ba9c89aaf218'
# very bad video, strange that it's ok in panoramas
# 908599515-1584450216985-kgmPW	https://autoru-panorama-internal.s3.mds.yandex.net/upload/71e/024/71e024f6-5b4d-48b6-9755-ba9c89aaf218
# very bad, but can't find on service,
# https://auto.ru/cars/used/sale/dodge/caliber/1096883676-73493883/
# better to make another

path = 'images_from_video/videos_ok_5c8_a96_5c8a9690-0456-4b54-8c18-a26e706f5566'

# fixed share of screen, small probabilites for trucks
# ok frames: 172
# bad frames without class: 4
# bad boxes: 4
# bad boxes outside of image: 0

path = 'images_from_video/videos_ok_5c8_a96_5c8a9690-0456-4b54-8c18-a26e706f5566'
# truck - partially fixed with bouding boxes todo: smaller probabilities for trucks
# ok frames: 169
# bad frames without class: 9
# bad boxes: 9
# bad boxes outside of image: 0

path = 'images_from_video/videos_ok_33c_9a7_33c9a73e-77c3-48d1-99b6-ab15197c31b7'
# fixed by bounding box fixes # car instead of truck , so still 4 errors
# ok frames: 197
# bad frames without class: 4
# bad boxes: 4
# bad boxes outside of image: 0

path = 'images_from_video/videos_ok_9a5_447_9a544719-3d6a-4c49-9777-c6ff71f51f4c'
# fixed by bouding box fixes # smaller probabilites for trucks
# ok frames: 185
# bad frames without class: 13
# bad boxes: 13
# bad boxes outside of image: 0



path = 'images_from_video/videos_ok_896_b4f_896b4fdf-1d2d-4905-9247-19ea0404e7f6'
# {'url': 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/896/b4f/896b4fdf-1d2d-4905-9247-19ea0404e7f6',
#  'panorama_status': 'completed',
#  'image_path': 'images_from_video/videos_ok_896_b4f_896b4fdf-1d2d-4905-9247-19ea0404e7f6',
#  'classifier_result': 'BAD',
#  'total_analyzed_frames': 173,
#  'big_vehicle_middle_screen_found': 149,
#  'no_big_vehicle_middle_screen_found': 24,
#  'car_found_but_bounding_box_unsafe_margin': 5,
#  'bounding_boxes_outside_image': 3}
# not fixed: suitcase, bottle, cell_phone and low probability (39-47%)

path = 'images_from_video/videos_ok_618_0ac_6180ac75-f84e-4f97-a93a-cd5ab073ffc1'
# fixed with lower thresholds


#### ok classifier
path = 'images_from_video/videos_ok_46a_061_46a061a5-b242-49b1-a41e-65f13b003784'
# outside margin
# {'url': 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/46a/061/46a061a5-b242-49b1-a41e-65f13b003784',
#  'panorama_status': 'completed',
#  'image_path': 'images_from_video/videos_ok_46a_061_46a061a5-b242-49b1-a41e-65f13b003784',
#  'classifier_result': 'GOOD',
#  'total_analyzed_frames': 106,
#  'big_vehicle_middle_screen_found': 106,
#  'no_big_vehicle_middle_screen_found': 0,
#  'car_found_but_bounding_box_unsafe_margin': 40,
#  'bounding_boxes_outside_image': 12}

path = 'images_from_video/videos_ok_a48_59c_a4859c48-00e9-41f6-b0d1-20ab03648fa4'
# {'url': 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/a48/59c/a4859c48-00e9-41f6-b0d1-20ab03648fa4',
#  'panorama_status': 'completed',
#  'image_path': 'images_from_video/videos_ok_a48_59c_a4859c48-00e9-41f6-b0d1-20ab03648fa4',
#  'classifier_result': 'GOOD',
#  'total_analyzed_frames': 81,
#  'big_vehicle_middle_screen_found': 81,
#  'no_big_vehicle_middle_screen_found': 0,
#  'car_found_but_bounding_box_unsafe_margin': 80,
#  'bounding_boxes_outside_image': 45}

path = 'images_from_video/videos_ok_543_1cf_5431cfb5-f14f-474e-becd-c83b6308e132'
# trash
# 'url': 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/543/1cf/5431cfb5-f14f-474e-becd-c83b6308e132',
#  'panorama_status': 'completed',
#  'image_path': 'images_from_video/videos_ok_543_1cf_5431cfb5-f14f-474e-becd-c83b6308e132',
#  'classifier_result': 'GOOD',
#  'total_analyzed_frames': 89,
#  'big_vehicle_middle_screen_found': 89,
#  'no_big_vehicle_middle_screen_found': 0,
#  'car_found_but_bounding_box_unsafe_margin': 51,
#  'bounding_boxes_outside_image': 33}


path = 'images_from_video/videos_ok_b60_0a1_b600a1f3-5e1d-4b06-ab7d-eeb43e9e3596'
# 'url': 'https://autoru-panorama-internal.s3.mds.yandex.net/upload/b60/0a1/b600a1f3-5e1d-4b06-ab7d-eeb43e9e3596',
#  'panorama_status': 'completed',
#  'image_path': 'images_from_video/videos_ok_b60_0a1_b600a1f3-5e1d-4b06-ab7d-eeb43e9e3596',
#  'classifier_result': 'GOOD',
#  'total_analyzed_frames': 110,
#  'big_vehicle_middle_screen_found': 110,
#  'no_big_vehicle_middle_screen_found': 0,
#  'car_found_but_bounding_box_unsafe_margin': 31,
#  'bounding_boxes_outside_image': 17}

# check failed < 0.05
# seems ok
# for path in ['images_from_video/videos_failed_366_9e2_3669e234-3ad7-4551-87cb-e5e3f0237550',
#  'images_from_video/videos_failed_120_5bc_1205bc7a-17c7-47a1-9f5f-159068136043',
#  'images_from_video/videos_failed_da3_777_da37771d-93a8-4496-88bb-f5318ab34347',
#  'images_from_video/videos_failed_b5c_7f9_b5c7f9a6-0d70-4d57-af95-4a84ce05bacc',
#  'images_from_video/videos_failed_159_49f_15949f2b-cfa8-4b9b-bd72-d405d136edc0']:
    

# check failed > 0.15 (all relly failed)
# for path in ['images_from_video/videos_failed_596_adb_596adbd6-3fba-407c-a2aa-da1cd9649d09',
#  'images_from_video/videos_failed_07d_373_07d37364-2f84-486a-a3ea-0dc96873edad',
#  'images_from_video/videos_failed_79f_75f_79f75fec-9c3a-43db-b392-9ddad58bf3b0',
#  'images_from_video/videos_failed_886_f72_886f7275-e49e-4d10-be19-f72180788dea',
#  'images_from_video/videos_failed_c96_89e_c9689eec-9017-4036-9e84-ae430318a5b0']:
#     check_video(file_path = '', make_frames = False, 
#                             debug_all = False, 
#                             debug_filter_big_boxes = False, folder_path = path,
#                             debug_margins = True,
#                             print_bad_photo=True,
#                             debug_share_of_screen=True
#                            )


# check ok > 0.15
for path in ['images_from_video/videos_ok_53b_d6a_53bd6a46-5540-4428-a879-4d4060f15ad7',
 'images_from_video/videos_ok_62d_768_62d768ce-077c-471d-bf25-22cb5cf6767d',
 'images_from_video/videos_ok_7f8_75a_7f875a60-8493-4224-a56d-8120557006a3',
 'images_from_video/videos_ok_fab_258_fab258d8-faa8-4fd7-a6bd-1e87f8b5b37b',
 'images_from_video/videos_ok_c92_274_c92274ba-0528-4ece-b520-6f9abd3c1a7d',
 'images_from_video/videos_ok_543_1cf_5431cfb5-f14f-474e-becd-c83b6308e132',
 'images_from_video/videos_ok_618_0ac_6180ac75-f84e-4f97-a93a-cd5ab073ffc1',
 'images_from_video/videos_ok_47b_fff_47bfff50-9ce7-4bc0-8d21-96a7019cbe4b',
 'images_from_video/videos_ok_896_b4f_896b4fdf-1d2d-4905-9247-19ea0404e7f6',
 'images_from_video/videos_ok_b60_0a1_b600a1f3-5e1d-4b06-ab7d-eeb43e9e3596',
 'images_from_video/videos_ok_265_f80_265f80ff-a11a-4c15-b520-5b8aa870f489',
 'images_from_video/videos_ok_dd3_cc1_dd3cc1ef-af5b-4e89-98c4-50dc454e9c72',
 'images_from_video/videos_ok_f1b_f0a_f1bf0a37-8193-44ed-9605-da1bd29fd043',
 'images_from_video/videos_ok_117_07f_11707fe0-ffa6-46bc-8d45-99a957edd890',
 'images_from_video/videos_ok_a48_59c_a4859c48-00e9-41f6-b0d1-20ab03648fa4',
 'images_from_video/videos_ok_8f7_73e_8f773e27-71b4-4392-badd-c7fdc2bbf091',
 'images_from_video/videos_ok_20b_eda_20beda0a-d143-4d48-a9e5-4c75f3755ffb',
 'images_from_video/videos_ok_485_00e_48500e30-3e82-4a54-876e-e133a787fc59',
 'images_from_video/videos_ok_7a6_3e4_7a63e421-742f-43b0-ab9a-6e6909bb4764',
 'images_from_video/videos_ok_9a5_447_9a544719-3d6a-4c49-9777-c6ff71f51f4c',
 'images_from_video/videos_ok_598_223_5982233c-b414-4968-93ab-226aff952573',
 'images_from_video/videos_ok_491_ef0_491ef0e4-fee9-4ec3-9411-544120739f30',
 'images_from_video/videos_ok_5c8_a96_5c8a9690-0456-4b54-8c18-a26e706f5566',
 'images_from_video/videos_ok_528_6e0_5286e0a3-ee9d-4793-a346-8b236d6d2f5d',
 'images_from_video/videos_ok_362_974_362974b0-5e49-4eea-b780-ce13afb6f05c',
 'images_from_video/videos_ok_f1a_1f2_f1a1f26f-7cf2-4275-af56-616fbac718af',
 'images_from_video/videos_ok_889_af7_889af702-ad32-43d8-a21a-7ca48d8dfef3',
 'images_from_video/videos_ok_bbb_a13_bbba139b-2f39-4810-b1e8-b01e3ae221d3',
 'images_from_video/videos_ok_221_052_221052db-f985-432e-b07e-6d077a33b191',
 'images_from_video/videos_ok_e07_0c5_e070c5ff-09b3-4715-b6b5-fdbc60970ba3',
 'images_from_video/videos_ok_ee1_ef4_ee1ef438-b983-4a9b-95f0-3a421ebda309',
 'images_from_video/videos_ok_31a_46f_31a46f74-62a2-417a-bd3e-a30bb9c37d8a',
 'images_from_video/videos_ok_71e_024_71e024f6-5b4d-48b6-9755-ba9c89aaf218']:
    check_video(file_path = '', make_frames = False, 
                            debug_all = False, 
                            debug_filter_big_boxes = False, folder_path = path,
                            debug_margins = True,
                            print_bad_photo=True,
                            debug_share_of_screen=True
                           )


    

# ok frames: 226
# bad frames without class: 0
# bad boxes: 0
# bad boxes outside of image: 0
# path = ''

# 739975341-1584107375522-oSkJg	https://autoru-panorama-internal.s3.mds.yandex.net/upload/043/3a0/0433a0f0-544c-4d25-8448-0fab3b0b72bf
# 1066934494-1583935918687-qdKtz	https://autoru-panorama-internal.s3.mds.yandex.net/upload/33c/9a7/33c9a73e-77c3-48d1-99b6-ab15197c31b7
# 920854619-1583914538433-CWDrk	https://autoru-panorama-internal.s3.mds.yandex.net/upload/5c8/a96/5c8a9690-0456-4b54-8c18-a26e706f5566
# 1130504595-1584455994055-9PyHe	https://autoru-panorama-internal.s3.mds.yandex.net/upload/618/0ac/6180ac75-f84e-4f97-a93a-cd5ab073ffc1

# 908599515-1584450216985-kgmPW	https://autoru-panorama-internal.s3.mds.yandex.net/upload/71e/024/71e024f6-5b4d-48b6-9755-ba9c89aaf218
# very bad, but can't find on service,
# https://auto.ru/cars/used/sale/dodge/caliber/1096883676-73493883/
# better to make another

# 1133125868-1583858493084-sNI8D	https://autoru-panorama-internal.s3.mds.yandex.net/upload/896/b4f/896b4fdf-1d2d-4905-9247-19ea0404e7f6
# 857678021-1583848639251-XqMBx	https://autoru-panorama-internal.s3.mds.yandex.net/upload/9a5/447/9a544719-3d6a-4c49-9777-c6ff71f51f4c

#	category_full	id	is_active	panorama_id	was_active	create_date
# 0	"CARS"	"1096883676-73493883"	true	"908599515-1584450216985-kgmPW"	true	2020-03-17
# 1	"ARTIC"	"16226196-5a95948f"	true	"920854619-1583914538433-CWDrk"	true	2019-11-29
# 2	"ARTIC"	"16265218-c704d1c6"	true	"739975341-1584107375522-oSkJg"	true	2019-12-27
# 3	"ARTIC"	"16156280-055fbc32"	false	"857678021-1583848639251-XqMBx"	true	2019-10-15


In [ ]:
import IPython.display
# IPython.display.Image('images_from_video/videos_failed_366_9e2_3669e234-3ad7-4551-87cb-e5e3f0237550/1.jpg') ok
# 'images_from_video/videos_failed_120_5bc_1205bc7a-17c7-47a1-9f5f-159068136043',
#  'images_from_video/videos_failed_da3_777_da37771d-93a8-4496-88bb-f5318ab34347',
#  'images_from_video/videos_failed_b5c_7f9_b5c7f9a6-0d70-4d57-af95-4a84ce05bacc',
#  'images_from_video/videos_failed_159_49f_15949f2b-cfa8-4b9b-bd72-d405d136edc0'

res = []
for prefix in ['images_from_video/videos_failed_366_9e2_3669e234-3ad7-4551-87cb-e5e3f0237550',
 'images_from_video/videos_failed_120_5bc_1205bc7a-17c7-47a1-9f5f-159068136043',
 'images_from_video/videos_failed_da3_777_da37771d-93a8-4496-88bb-f5318ab34347',
 'images_from_video/videos_failed_b5c_7f9_b5c7f9a6-0d70-4d57-af95-4a84ce05bacc',
 'images_from_video/videos_failed_159_49f_15949f2b-cfa8-4b9b-bd72-d405d136edc0']:
    
    path = prefix + "/1.jpg"
#     path = prefix + "/400.jpg"
#     path = prefix + "/800.jpg"
    res.append(IPython.display.Image(path))

In [ ]:
# res[5]

In [ ]:
df_results['share_no_big_vehicle_middle_screen_found'] = df_results['no_big_vehicle_middle_screen_found']/df_results['total_analyzed_frames']

In [ ]:
df_results['share_car_found_but_bounding_box_unsafe_margin'] = df_results['car_found_but_bounding_box_unsafe_margin']/(df_results['total_analyzed_frames'] - df_results['no_big_vehicle_middle_screen_found'])

In [ ]:
df_results['share_bad_frames'] = (df_results['no_big_vehicle_middle_screen_found'] + df_results['car_found_but_bounding_box_unsafe_margin'])/df_results['total_analyzed_frames']

In [ ]:
df_results['share_really_bad_frames'] = (df_results['no_big_vehicle_middle_screen_found'] + df_results['bounding_boxes_outside_image'])/df_results['total_analyzed_frames']

In [ ]:
df_results['bad_frames'] = (df_results['no_big_vehicle_middle_screen_found'] + df_results['car_found_but_bounding_box_unsafe_margin'])
df_results['really_bad_frames'] = (df_results['no_big_vehicle_middle_screen_found'] + df_results['bounding_boxes_outside_image'])

In [ ]:
list(df_results[(df_results.classifier_result == BAD) & (df_results.panorama_status == COMPLETED)].url)

In [ ]:
# 739975341-1584107375522-oSkJg	https://autoru-panorama-internal.s3.mds.yandex.net/upload/043/3a0/0433a0f0-544c-4d25-8448-0fab3b0b72bf
# 1066934494-1583935918687-qdKtz	https://autoru-panorama-internal.s3.mds.yandex.net/upload/33c/9a7/33c9a73e-77c3-48d1-99b6-ab15197c31b7
# 920854619-1583914538433-CWDrk	https://autoru-panorama-internal.s3.mds.yandex.net/upload/5c8/a96/5c8a9690-0456-4b54-8c18-a26e706f5566
# 1130504595-1584455994055-9PyHe	https://autoru-panorama-internal.s3.mds.yandex.net/upload/618/0ac/6180ac75-f84e-4f97-a93a-cd5ab073ffc1

# 908599515-1584450216985-kgmPW	https://autoru-panorama-internal.s3.mds.yandex.net/upload/71e/024/71e024f6-5b4d-48b6-9755-ba9c89aaf218
# very bad, but can't find on service,
# https://auto.ru/cars/used/sale/dodge/caliber/1096883676-73493883/
# better to make another

# 1133125868-1583858493084-sNI8D	https://autoru-panorama-internal.s3.mds.yandex.net/upload/896/b4f/896b4fdf-1d2d-4905-9247-19ea0404e7f6
# 857678021-1583848639251-XqMBx	https://autoru-panorama-internal.s3.mds.yandex.net/upload/9a5/447/9a544719-3d6a-4c49-9777-c6ff71f51f4c

#	category_full	id	is_active	panorama_id	was_active	create_date
# 0	"CARS"	"1096883676-73493883"	true	"908599515-1584450216985-kgmPW"	true	2020-03-17
# 1	"ARTIC"	"16226196-5a95948f"	true	"920854619-1583914538433-CWDrk"	true	2019-11-29
# 2	"ARTIC"	"16265218-c704d1c6"	true	"739975341-1584107375522-oSkJg"	true	2019-12-27
# 3	"ARTIC"	"16156280-055fbc32"	false	"857678021-1583848639251-XqMBx"	true	2019-10-15

In [ ]:
df_results.share_no_big_vehicle_middle_screen_found.describe()

In [ ]:
df_results[(df_results.panorama_status == COMPLETED)].share_no_big_vehicle_middle_screen_found.describe()

In [ ]:
df_results[(df_results.panorama_status == COMPLETED)].no_big_vehicle_middle_screen_found.max()

In [ ]:
df_results[(df_results.panorama_status == COMPLETED)].car_found_but_bounding_box_unsafe_margin.max()

In [ ]:
df_results[(df_results.panorama_status == COMPLETED)].bounding_boxes_outside_image.max()

In [ ]:
df_results[(df_results.panorama_status == COMPLETED)].total_analyzed_frames.min()

In [ ]:
df_results[(df_results.panorama_status == FAILED)].share_no_big_vehicle_middle_screen_found.describe()

### Analyse strange cases

In [ ]:
# images_from_video/videos_ok_287_b38_287b380e-e940-44d1-bf0d-e4e1401dd015
# not ok frame: printing
# debugging photo: images_from_video/videos_ok_287_b38_287b380e-e940-44d1-bf0d-e4e1401dd015/1729.jpg
# not ok frame: printing
# debugging photo: images_from_video/videos_ok_287_b38_287b380e-e940-44d1-bf0d-e4e1401dd015/1753.jpg
# not ok frame: printing
# debugging photo: images_from_video/videos_ok_287_b38_287b380e-e940-44d1-bf0d-e4e1401dd015/1765.jpg

path = 'images_from_video/1.mp4/121.jpg'
path = 'images_from_video/videos_ok_03d_04a_03d04ae5-1954-426c-b6ed-2c997f3ddcca/445.jpg'
# train

path = 'images_from_video/videos_ok_03d_04a_03d04ae5-1954-426c-b6ed-2c997f3ddcca/457.jpg'
# parking meter + car 33%

path = 'images_from_video/videos_ok_03d_04a_03d04ae5-1954-426c-b6ed-2c997f3ddcca/469.jpg'
# car 47% - todo fix

path = 'images_from_video/videos_ok_31e_57f_31e57f3e-a6fb-475f-a204-0fd9b8250be4/265.jpg'
# check_big_car_tflite_results(path, debug_all=True, debug_filter_big_boxes=True, debug_bounding_box_margin=True)

path = 'images_from_video/videos_ok_46a_061_46a061a5-b242-49b1-a41e-65f13b003784/1093.jpg'

path = 'images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1273.jpg'
# bounding box not ok: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1261.jpg
# no car/truck/bus found: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1273.jpg
# bounding box not ok: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1273.jpg
# no car/truck/bus found: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1285.jpg
# bounding box not ok: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1285.jpg
# no car/truck/bus found: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1297.jpg
# bounding box not ok: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1297.jpg
# no car/truck/bus found: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1309.jpg
# bounding box not ok: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1309.jpg
# no car/truck/bus found: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1321.jpg
# bounding box not ok: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1321.jpg
# no car/truck/bus found: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1333.jpg
# bounding box not ok: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/1333.jpg
# big box not in the center found: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/445.jpg
# no car/truck/bus found: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/445.jpg
# bounding box not ok: images_from_video/videos_ok_dad_56f_dad56ff5-34be-4a46-afb7-22962a9b37df/445.jpg'
check_big_car_tflite_results(path, debug_all=True, debug_filter_big_boxes=False, debug_bounding_box_margin=True,
                            debug_share_of_screen=True)